In [172]:
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [151]:
engine = create_engine("sqlite:///data/mta_turnstile.db")

In [152]:
all_tables = engine.table_names()
#all_tables

<ipython-input-152-454fd37e24c5>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  all_tables = engine.table_names()


In [153]:
mtadf = pd.read_sql('SELECT * FROM turnstile_2019_0105;',engine)
mtadf.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/29/2018,03:00:00,REGULAR,6889287,2335920
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/29/2018,07:00:00,REGULAR,6889299,2335936
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/29/2018,11:00:00,REGULAR,6889364,2336038
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/29/2018,15:00:00,REGULAR,6889605,2336101
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/29/2018,19:00:00,REGULAR,6889966,2336173


In [154]:
print('DTYPES:\n',mtadf.dtypes,'\n')
print(mtadf.columns)

DTYPES:
 C/A                                                                     object
UNIT                                                                    object
SCP                                                                     object
STATION                                                                 object
LINENAME                                                                object
DIVISION                                                                object
DATE                                                                    object
TIME                                                                    object
DESC                                                                    object
ENTRIES                                                                  int64
EXITS                                                                    int64
dtype: object 

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',


In [155]:
# fix the insane amount of space after 'EXITS'
mtadf.rename(str.strip, axis='columns',inplace=True)
mtadf.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [156]:
# change DATE and TIME to datetime objects
mtadf['DATE']=pd.to_datetime(mtadf['DATE'])
mtadf['TIME']=pd.to_datetime(mtadf['TIME'],format='%H:%M:%S')

In [157]:
# create column for HOUR and MINUTE
mtadf['HOUR']=mtadf['TIME'].dt.hour
mtadf['MIN']=mtadf['TIME'].dt.minute

# drop the TIME column
mtadf.drop(columns='TIME',axis=1,inplace=True)

In [159]:
# create unique identifyer for each turnstile then drop original columns
mtadf['TURNSTILE'] = mtadf['C/A'] + '-' + mtadf['UNIT'] + '-' + mtadf['SCP']
mtadf.drop(['C/A','UNIT','SCP'],axis=1,inplace=True)

# rearrange order of columns
col_names = ['STATION','TURNSTILE','DATE','HOUR','MIN','ENTRIES','EXITS','LINENAME','DIVISION','DESC']
mtadf = mtadf.reindex(columns=col_names)

In [237]:
# check for null values in dataset -- no null values
mtadf.isna().sum()

STATION      0
TURNSTILE    0
DATE         0
HOUR         0
MIN          0
ENTRIES      0
EXITS        0
LINENAME     0
DIVISION     0
DESC         0
dtype: int64

In [236]:
# sort the database by station, then turnstile, then date, then time
mtadfs = mtadf.sort_values(['STATION','TURNSTILE','DATE','HOUR','MIN'])

# which 10 stations have the least amount of turnstiles?
mtadfs.groupby(['STATION'])['TURNSTILE'].nunique().nsmallest(10)
mtadfs_leastt = mtadfs[mtadfs['STATION'].isin(['138/GRAND CONC','182-183 STS','190 ST'])
                       &mtadfs['DATE'].isin(['2019-01-04'])]

# max amount of entries per station
#CLEVELAND ST and SUTTER AV have significantly less entries
mtadfs_leastt.groupby(['STATION'])['ENTRIES'].max()

STATION
138/GRAND CONC     991352029
182-183 STS       1476159278
190 ST            1442503008
Name: ENTRIES, dtype: int64

In [164]:
#mtadf.head()
#explore_data['C/A'].nunique() #737
#explore_data['UNIT'].nunique() #465
#explore_data['SCP'].nunique() #219
#explore_data['STATION'].nunique() #377

,HOUR,MIN,ENTRIES,EXITS
count,317.000000,317.0,3.170000e+02,3.170000e+02
mean,13.006309,0.0,7.873335e+07,4.496047e+07
std,6.851958,0.0,3.198354e+08,1.660011e+08
min,3.000000,0.0,0.000000e+00,0.000000e+00
25%,7.000000,0.0,1.807684e+06,1.346728e+06
50%,15.000000,0.0,6.039640e+06,3.622921e+06
75%,19.000000,0.0,8.798028e+06,6.477034e+06
max,23.000000,0.0,2.029932e+09,7.357994e+08
